**INSTALLING LIBRARIES**

In [3]:
!pip install langchain
!pip install langchain_huggingface
!pip install langchain-community
!pip install langchain-qdrant
!pip install transformers
!pip install pypdf
!pip install qdrant_client
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 37.2 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.6
    Uninstalling google-ai-generativelanguage-0.6.6:
      Successfully uninstalled google-ai-generativelanguage-0.6.6
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.7.2
    Uninstalling google-generativeai-0.7.2:
      Successfully uninstalled google-generativeai-0.7.2


**IMPORTING LIBRARIES**

In [5]:
from qdrant_client import QdrantClient
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

**QDRANT VECTOR STORE**

The QdrantClient is initialized to interact with the Qdrant cloud-based vector store using an API key.
The collection where embeddings will be stored is named "Ethics_of_Islam."
Embedding

In [6]:
# setting up qdrant vector store to store the embedding generated by embedding model
qdrant_url = "url here"
qdrant_key = "api key here"
collection_name = "Ethics_of_Islam"

In [7]:
api_key="google api key here"

 **LLM**

In [8]:
llm_name = "gemini-1.5-pro"

**EMBEDDING MODEL**

The script initializes an embedding model (BAAI/bge-small-en-v1.5) from Hugging Face to convert document text into vectors for storage and retrieval.

In [9]:
# Initialize embedding model with "BAAI/bge-small-en-v1.5"
embed_model= HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

<ipython-input-9-2b6da3141ede>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model= HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**LOADING THE DOCUMENT**

A PDF file (Ethics_of_Islam.pdf) is loaded using PyPDFLoader to extract its content.


In [10]:
loader= PyPDFLoader("/content/Ethics_of_Islam.pdf")

pages= loader.load()

In [11]:
len(pages) #used len funtion to check the total pages in the above doc

240

**SPLITTING THE DOCUMENT**

The loaded document is split into smaller chunks of 1500 characters with a 150-character overlap for easier processing.


In [12]:
#splitting the documents into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500, #size of the chunk
    chunk_overlap = 150 #Defines the number of characters that will overlap between consecutive chunks.Here, 150 characters from the end of one chunk will appear at the start of the next chunk.
)

In [13]:
splits= text_splitter.split_documents(pages)

In [14]:
len(splits) #used len funtion to check the number of splits

556

**EMBEDDING AND STORAGE:**

The document chunks are embedded using the Hugging Face model and stored in the Qdrant vector store. The force_recreate=True flag ensures the collection is recreated with new vector dimensions.

In [17]:
qdrant = QdrantVectorStore.from_documents(
    splits,
    embed_model, # Embedding model used to convert documents into vectors
    url=qdrant_url, # URL for the Qdrant service
    prefer_grpc=True, # Prefer gRPC over HTTP for communication
    api_key=qdrant_key, # API key for accessing the Qdrant service
    collection_name=collection_name, # Name of the collection to store the vectors in
    force_recreate=True  # This will recreate the collection with the new vector dimensions
)

**CHAIN IMPLEMENTATION**

A prompt template is created to guide the language model to answer user queries based on retrieved context from the document.
A retriever is set up to fetch 3 relevant document chunks based on similarity to the user’s query.

In [18]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

The function format_docs(docs) is designed to take a list of document objects (docs) and format their content into a single string, with each document's content separated by two newline characters ("\n\n").

In [29]:
prompt_str="""
Answer the user question based only on the following context:
{context}

Question: {question}
""" #defines the template for the prompt that will be sent to the language model (LLM)

_prompt = ChatPromptTemplate.from_template(prompt_str) #designed to guide the LLM to answer based on the retrieved context.
num_chunks=3 # 3 chunks will be fetched for each query.
retriever = qdrant.as_retriever(search_type="similarity",
                                        search_kwargs={"k": num_chunks}) #sets up a retriever from the Qdrant vector database, which will be responsible for fetching the most relevant documents or chunks based on similarity to the user's question.
chat_llm = ChatGoogleGenerativeAI(model=llm_name, google_api_key=api_key, temperature=0)
query_fetcher= itemgetter("question") #It is used to extract the user's question from the input

setup={"question":query_fetcher,"context":query_fetcher | retriever | format_docs} #A dictionary that defines how to prepare the inputs for the LLM

_chain = (setup |_prompt | chat_llm)

In [30]:
query="what are the ethical issues in Islam?"

In [31]:
response=_chain.invoke({"question":query})

In [32]:
response.content

"This excerpt lists two main categories of Islamic ethics:\n\n1. **Invariable personal behavior (’ilm al-akhlâq):** This category focuses on inherent character traits that are considered good or bad regardless of social context. Examples include gentleness, generosity, and having a sense of shame.\n2. **Interpersonal ethics:** This category, which the excerpt begins to describe but does not define in detail, focuses on a person's behavior towards others. \n\nThe passage continues by mentioning a third category related to acquiring knowledge and understanding the purpose of ethical actions. However, it does not elaborate on the specifics of this category or provide further details about the second category of interpersonal ethics. \n"

**WORKFLOW SUMMARY:**

The script loads a PDF, splits it into chunks, stores the embeddings in a Qdrant vector store, retrieves the most relevant document chunks based on a query, and uses a Google LLM to generate a response based on the retrieved context.